<a href="https://colab.research.google.com/github/axwf/ffmpeg-upscale-colab/blob/main/ffmpeg_upscale.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## FFmpeg Upscaler
- Requirements:
  - a Google Drive
  - 4-6 GB of avalible space in your Google Drive + additional storage for video(s)
  - GPU compute or CPU runtime
- **uncheck** `FIRST_LAUNCH` and `INSTALL_FFMPEG` **after** first launch


In [ ]:
#@markdown ## 1) *Setup*
#@markdown *** This may take up a long time (30+ minutes).**
#@markdown - **ALWAYS mounts drive**
#@markdown - `FIRST_LAUNCH` creates FFmpegUpscale and other additional folders in your google drive
#@markdown - `INSTALL_FFMPEG` installs [FFmpeg with NVIDIA Acceleration](https://docs.nvidia.com/video-technologies/video-codec-sdk/12.0/ffmpeg-with-nvidia-gpu/index.html)

import os

from pathlib import Path

OPTIONS = {}

FIRST_LAUNCH = True #@param {type:"boolean"}
INSTALL_FFMPEG = True #@param {type:"boolean"}
OPTIONS['FIRST_LAUNCH'] = FIRST_LAUNCH
OPTIONS['INSTALL_FFMPEG'] = INSTALL_FFMPEG


!echo "-= Mounting Google Drive =-"
%cd /

from google.colab import drive
drive.mount('/content/drive')


if OPTIONS['FIRST_LAUNCH']:
  if os.path.exists('/content/drive/MyDrive/FFmpegUpscale'):
    print("Directory Already Exists! Deselect `FIRST_LAUNCH`")
  else:
    # >> create FFmpegUpscale DIR if first launch
    os.mkdir('/content/drive/MyDrive/FFmpegUpscale')
    os.mkdir('/content/drive/MyDrive/FFmpegUpscale/inputVideosHere')
    os.mkdir('/content/drive/MyDrive/FFmpegUpscale/output')

    %cd /content/drive/MyDrive/FFmpegUpscale

if OPTIONS['INSTALL_FFMPEG']:

  if not os.path.exists('/content/drive/MyDrive/FFmpegUpscale'):
    print("Missing Directory! Select `FIRST_LAUNCH`")
  else:
    %cd /content/drive/MyDrive/FFmpegUpscale

    # >> download & install ffnvcodec
    !echo '-= Downloading & Installing =-' && git clone https://git.videolan.org/git/ffmpeg/nv-codec-headers.git
    !cd nv-codec-headers && sudo make install && cd -

    # >> download ffmpeg
    !echo '-= Downloading ffmpeg =-' && git clone https://git.ffmpeg.org/ffmpeg.git ffmpeg/

    # >> installing required packages
    !sudo apt-get install build-essential yasm cmake libtool libc6 libc6-dev unzip wget libnuma1 libnuma-dev
    !sudo apt-get install nasm
    !apt-get install -y cuda

    os.environ['PATH'] += ':/usr/local/cuda/bin'
    os.environ['CUDA_HOME'] = '/usr/local/cuda'

    # >> configure
    !chmod +x /content/drive/MyDrive/FFmpegUpscale/ffmpeg/configure
    !cd /content/drive/MyDrive/FFmpegUpscale/ffmpeg/ && /content/drive/MyDrive/FFmpegUpscale/ffmpeg/configure --enable-nonfree --enable-cuda-nvcc --enable-libnpp --extra-cflags=-I/usr/local/cuda/include --extra-ldflags=-L/usr/local/cuda/lib64 --disable-static --enable-shared

    # >> compile
    !chmod +x /content/drive/MyDrive/FFmpegUpscale/ffmpeg/ffbuild/version.sh
    !cd /content/drive/MyDrive/FFmpegUpscale/ffmpeg/ && sudo make -j 8 NVCC=/usr/local/cuda/bin/nvcc

    # >> install lib
    !chmod +x /content/drive/MyDrive/FFmpegUpscale/ffmpeg/ffbuild/pkgconfig_generate.sh
    !cd /content/drive/MyDrive/FFmpegUpscale/ffmpeg/ && sudo make install

WORKSPACE = "/content/drive/MyDrive/FFmpegUpscale"
%cd $WORKSPACE

!echo "-= Done! =-"

In [ ]:
#@markdown ## 2) *Select Video*
#@markdown - upload your video to your google drive in `FFmpegUpscale/inputVideosHere`
#@markdown - copy the file name **WITH EXTENSION** and paste into `video_name`

video_name = 'file name here.video extension'#@param{type: "string"}
file_name = video_name.split('/')[-1]
video_path = f'/content/drive/MyDrive/FFmpegUpscale/inputVideosHere/{video_name}'
if not os.path.exists(video_path):
  print("File Not Found!")
else:
  print("Video set to:\n",video_path)

In [ ]:
#@markdown ## 3) *Upscale Video*
#@markdown - note: only 8 bit video is supported
#@markdown - select the resolution to upscale to in `resolution`
#@markdown - will always use `h264_nvenc` codec for encoding

import cv2 as cv

# >> set ffmpeg path
!sudo mv /content/drive/MyDrive/FFmpegUpscale/ffmpeg /usr/local/bin/ffmpeg

video = cv.VideoCapture(video_path)
video_width = int(video.get(cv.CAP_PROP_FRAME_WIDTH))
video_height = int(video.get(cv.CAP_PROP_FRAME_HEIGHT))

convert_to_8_bit = False #@param {type:"boolean"}
encoder = "h264_nvenc" #@param ["h264_nvenc", "libx264", "hevc_nvenc"] {type:"string"}
resolution = "2k (2560 x 1440)" #@param ["FHD (1920 x 1080)", "2k (2560 x 1440)", "4k (3840 x 2160)"] {type:"string"}
preset = "medium" #@param ["ultrafast", "superfast", "veryfast", "faster", "fast", "medium", "slow", "slower", "veryslow", "lossless"] {type:"string"}
profile = "main" #@param ["baseline", "main", "high", "high10", "high422", "high444"] {type:"string"}
cq = 30 #@param {type:"slider", min:0, max:63, step:1}
#@markdown - Lower = Better Quality (Slower)
#@markdown - Faster = Worse Quality (Faster)

output_name = 'upscaled_' + file_name

# >> output res
match resolution:
  case "FHD (1920 x 1080)":
    final_width=1920
    final_height=1080
  case "2k (2560 x 1440)":
    final_width=2560
    final_height=1440
  case "4k (3840 x 2160)":
    final_width=3840
    final_height=2160

os.environ['VIDEO_PATH'] = video_path
os.environ['ENCODER'] = encoder
os.environ['PRESET'] = preset
os.environ['PROFILE'] = profile
os.environ['CQ'] = str(cq)
os.environ['RESOLUTION'] = f'{final_width}:{final_height}'
os.environ['OUTPUT'] = f'/content/drive/MyDrive/FFmpegUpscale/output/{output_name}'

!export LD_LIBRARY_PATH=/content/drive/MyDrive/FFmpegUpscale/ffmpeg/libavdevice/libavdevice.so.:$LD_LIBRARY_PATH
!cd /content/drive/MyDrive/FFmpegUpscale/ffmpeg
print(f"Upscaling from {video_width}x{video_height} to {final_width}x{final_height}")
if  convert_to_8_bit:
  !ffmpeg -i "$VIDEO_PATH" -pix_fmt yuv420p -c:v "$ENCODER" -preset "$PRESET" -profile:v "$PROFILE" -cq "$CQ" -vf "scale=$RESOLUTION:flags=neighbor" -c:a copy "$OUTPUT"
else:
  !ffmpeg -i "$VIDEO_PATH" -c:v "$ENCODER" -preset "$PRESET" -profile:v "$PROFILE" -cq "$CQ" -vf "scale=$RESOLUTION:flags=neighbor" -c:a copy "$OUTPUT"

In [ ]:
#@markdown ## 3) *Update Nvidia Driver*
#@markdown

# prompt: update nvidia drivers to lts

# This code assumes you are using a CUDA-enabled runtime in Colab.
# It also assumes that you have already installed ffmpeg with NVidia acceleration as in the original code.
# If not, uncomment the relevant lines and run the installation section first.

!apt-get update
!apt-get install -y --no-install-recommends nvidia-driver-530 # Replace 530 with your desired LTS driver version
# You can find the available LTS drivers with: !apt-cache madison nvidia-driver
!nvidia-smi # Verify the driver version